<a href="https://colab.research.google.com/github/federicanotta01/BankUnitedTest/blob/main/TakeHomeExercise_Notta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Federica Notta property Feb 2024

In [ ]:
!git clone https://github.com/federicanotta01/BankUnitedTest

Cloning into 'BankUnitedTest'...
remote: Enumerating objects: 13, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 13 (delta 1), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (13/13), 4.45 KiB | 4.45 MiB/s, done.
Resolving deltas: 100% (1/1), done.


In [ ]:
# Import libraries
import pandas as pd
import numpy as np

In [ ]:
# Import Loan Positions data
LoanPositions = pd.read_csv('https://raw.githubusercontent.com/federicanotta01/BankUnitedTest/main/LoanPositions.csv')
# Import Loan Risk data
LoanRisk = pd.read_csv('https://raw.githubusercontent.com/federicanotta01/BankUnitedTest/main/LoanRisk.csv')

Check data

Loan Positions

In [ ]:
# print(LoanPositions.head())
print(LoanPositions.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   LoanNumber  10 non-null     int64
 1   Balance     10 non-null     int64
dtypes: int64(2)
memory usage: 288.0 bytes
None


Loan Risk

In [ ]:
# print(LoanRisk.head())
print(LoanRisk.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   LoanNumber  11 non-null     int64  
 1   PD          10 non-null     float64
 2   LGD         11 non-null     float64
dtypes: float64(2), int64(1)
memory usage: 392.0 bytes
None


In [ ]:
# Left join loanposition with loanrisk by "LoanNumber"
LoanData = pd.merge(LoanPositions, LoanRisk, on="LoanNumber", how="left")

In [ ]:
LoanData.head(5)

,LoanNumber,Balance,PD,LGD
0,1,1000000,0.01,0.1
1,2,2000000,0.01,0.1
2,3,3000000,0.01,0.1
3,4,4000000,0.01,0.1
4,5,5000000,0.01,0.1


Missing Values

In [ ]:
# Missing values in the dataframe
missing_values = LoanData.isna()

In [ ]:
# Sum missing bby columns
missing_counts = missing_values.sum()

In [ ]:
# Print the # of missing values in each column
print(missing_counts)

LoanNumber    0
Balance       0
PD            1
LGD           0
dtype: int64


In [ ]:
# Check variable distribution
print(LoanData['PD'].describe(percentiles=[.01, .25, .5, .75, .99]))

count    9.000000
mean     0.018889
std      0.017638
min      0.010000
1%       0.010000
25%      0.010000
50%      0.010000
75%      0.010000
99%      0.050000
max      0.050000
Name: PD, dtype: float64


In [ ]:
# Check for outliers -> As there are no outliers, it's reasonable to use the mean to input missing values.
# Since the max and min values are within a reasonable range -> No need to adjust max or min values.
# Additionally, the majority of the data (up to 75th percentile) is close to the mean-> this indicates that mean imputation will not significantly
# change the distribution
# Conclusion: mean imputation is a suitable approach

In [ ]:
# Fill missing values with the mean and save into a new DataFrame (M: after missing df)
LoanData_M = LoanData.fillna(LoanData.mean()).copy()

# Verify the filled_data DataFrame
print(LoanData_M.head())

   LoanNumber  Balance    PD  LGD
0           1  1000000  0.01  0.1
1           2  2000000  0.01  0.1
2           3  3000000  0.01  0.1
3           4  4000000  0.01  0.1
4           5  5000000  0.01  0.1


In [ ]:
# Check variable distribution after missing
print(LoanData_M['PD'].describe(percentiles=[.01, .25, .5, .75, .99]))

count    10.000000
mean      0.018889
std       0.016630
min       0.010000
1%        0.010000
25%       0.010000
50%       0.010000
75%       0.016667
99%       0.050000
max       0.050000
Name: PD, dtype: float64


In [ ]:
# Check variable miss after imputation
LoanData_M['PD'].isna().any()

False

Create a Random variable (RV) [0,1] & Flag variable that indicates 1 when RV < PD

In [ ]:
LoanData_M['RV'] = np.random.uniform(0, 1, len(LoanData_M))
LoanData_M['Flag'] = np.where(LoanData_M['RV'] < LoanData_M['PD'], 1, 0)
# 'Flag' indicates probability of loan default: 1 = higher, 0 = lower

In [ ]:
LoanData_M.head()

,LoanNumber,Balance,PD,LGD,RV,Flag
0,1,1000000,0.01,0.1,0.006588,1
1,2,2000000,0.01,0.1,0.739625,0
2,3,3000000,0.01,0.1,0.261047,0
3,4,4000000,0.01,0.1,0.194900,0
4,5,5000000,0.01,0.1,0.696568,0


In [ ]:
LoanData_M['Flag'].value_counts()

0    9
1    1
Name: Flag, dtype: int64

 Create another variable representing Flag * LGD * Balance

In [ ]:
LoanData_M['ExpectedLoss'] = LoanData_M['Flag'] * LoanData_M['LGD'] * LoanData_M['Balance']
# The variable 'ExpectedLoss' represents the expected loss for each loan.
# It is calculated by multiplying the probability of default (Flag),
# the loss given default (LGD), and the current balance (Balance).

Sum the variable 'ExpectedLoss' created in the prior step across all loans.

In [ ]:
total_expectedloss = LoanData_M['ExpectedLoss'].sum()
# 'total_expectedloss' represents the cumulative expected loss for all loans in the current portfolio.
# It provides an overall estimated of the total risk exposure associated with the loan portfolio.

In [ ]:
print(total_expectedloss)

100000.0


Create a loop that runs steps 3-5 10,000 times and stores the variable created in step 5 in a dataset. The final dataset should have 10,000 observations of the sum in step 5.

In [ ]:
simulation_loop = []
for _ in range(10000):
    LoanData_M['RV'] = np.random.uniform(0, 1, len(LoanData_M))
    LoanData_M['Flag'] = np.where(LoanData_M['RV'] < LoanData_M['PD'], 1, 0)
    LoanData_M['ExpectedLoss'] = LoanData_M['Flag'] * LoanData_M['LGD'] * LoanData_M['Balance']
    sum_variable = LoanData_M['ExpectedLoss'].sum()
    simulation_loop.append(sum_variable)

    # Convert the list to a DataFrame
LoanData_M_Simulation = pd.DataFrame(simulation_loop, columns=['Sum_Variable'])

In [ ]:
LoanData_M_Simulation.shape

(10000, 1)

In [ ]:
LoanData_M_Simulation.head()

,Sum_Variable
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0


Analyze the final dataset of 10,000 observations

In [ ]:
# Mean of the dataset
mean = LoanData_M_Simulation['Sum_Variable'].mean()
# 95th percentile of the dataset
p95 = LoanData_M_Simulation['Sum_Variable'].quantile(0.95)

In [ ]:
print(mean)
print(p95)

287560.0
2500000.0


In [116]:
# The mean represents the average value of the simulated variable across all 10,000 observations.
# It gives us an idea of the typical or expected value of the total expected loss.

# The 95th percentile represents the value below which 95% of the observations fall.

# The difference between the mean and the 95th percentile can happen due to the presence of outliers the dataset.

# To reduce the 95th percentile of the dataset, we can remove outliers or transform the data to make it less sensitive to extreme values.
# Also it would be better working with a bigger sample size.